In [7]:
import pandas as pd
import datetime
import glob
import os

all_store_list_files=sorted(glob.glob("/home/jian/BigLots/static_files/Store_list/*.txt"),reverse=True)

In [8]:
df_all_store_DMA=pd.DataFrame()
for file in all_store_list_files:
    df=pd.read_table(file,sep="|",dtype=str,usecols=["location_id","zip_cd"])
    df_all_store_DMA=df_all_store_DMA.append(df)
df_all_store_DMA=df_all_store_DMA.drop_duplicates("location_id")
df_all_store_DMA['zip_cd']=df_all_store_DMA['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

In [9]:
DMA_Mapping=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,usecols=[0,2],dtype=str)
DMA_Mapping=DMA_Mapping.rename(columns={"CODE":"zip_cd","NAME":"DMA"})


In [10]:
DMA_Mapping_first=DMA_Mapping.drop_duplicates("zip_cd")
DMA_Mapping_list=DMA_Mapping.groupby("zip_cd")['DMA'].apply(set).to_frame().reset_index().rename(columns={"DMA":"DMA_set"})

df_all_store_DMA=pd.merge(df_all_store_DMA,DMA_Mapping_first,on="zip_cd",how="left")
df_all_store_DMA=pd.merge(df_all_store_DMA,DMA_Mapping_list,on="zip_cd",how="left")

In [11]:
df_all_store_DMA.head(4)

,location_id,zip_cd,DMA,DMA_set
0,1,43232,"COLUMBUS, OH","{COLUMBUS, OH}"
1,29,45701,"COLUMBUS, OH","{COLUMBUS, OH}"
2,32,45419,DAYTON,{DAYTON}
3,43,43614,TOLEDO,{TOLEDO}


In [13]:
writer=pd.ExcelWriter("./all_store_dma.xlsx",engine="xlsxwriter")
df_all_store_DMA.to_excel(writer,"all_stores_DMA",index=False)

writer.save()